### コーパスの中身をkmeansでクラスタリング

In [1]:
from collections import defaultdict

from gensim import corpora
from gensim import models
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
model = KeyedVectors.load('model/word2vec_ramen_model.model')

In [3]:
max_vocab = 30000
vocab = list(model.wv.key_to_index.keys())[:max_vocab]
vectors = [model.wv[word] for word in vocab]

In [4]:
# クラスター数はこちらで任意の値を定める
n_clusters = 6
kmeans_model = KMeans(n_clusters=n_clusters, verbose=0, random_state=42)
kmeans_model.fit(vectors)

KMeans(n_clusters=6, random_state=42)

In [5]:
cluster_labels = kmeans_model.labels_
cluster_to_words = defaultdict(list)
for cluster_id, word in zip(cluster_labels, vocab):
    cluster_to_words[cluster_id].append(word)
    
for label, words in cluster_to_words.items():
    print(label, words[:20])

2 ['スープ', '美味しい', '醤油', '感じ', '濃厚', '煮干し', '旨味', '出汁', '味わい', 'つけ', '香り', '風味', '良く', '魚介', 'なく', 'バランス', '印象', '豚骨', '見た目', 'ストレート']
1 ['ラーメン', '0円', 'こと', 'よう', 'これ', 'そう', 'つけ麺', 'いい', 'ない', 'もの', 'そば', '特製', 'らーめん', '今回', 'それ', '好き', 'ところ', 'ここ', '味噌', '提供']
3 ['チャーシュー', '良い', 'メンマ', 'トッピング', 'ネギ', 'ワンタン', '食感', '野菜', '海苔', '具材', '旨い', 'ご飯', '美味い', '味付け', '調理', 'タイプ', 'シンプル', '玉子', '相性', 'ニンニク']
0 ['0分', '店内', 'カウンター', '食券', '注文', 'メニュー', '券売機', '到着', '購入', '店主', '待ち', '時間', '入店', '店員', '開店', '雰囲気', '着席', '先客', 'オーダー', 'お客さん']
5 ['さん', '0時', 'こちら', '訪問', '中華そば', '人気', '行列', '今日', 'ごちそうさま', '平日', 'ラーメン屋', '近く', 'ランチ', '営業', 'オープン', '以前', '評価', 'ラーメン店', '名店', '店舗']
4 ['徒歩', '場所', '食べログ', 'https', '.com', '出て', '土曜日', '外観', 'www', '沿い', '新宿', 'instagram', '方面', '路地', 'エリア', '営業時間', 'グルメ', 'jr', '駐車場', '新橋']


In [7]:
def change_dict_key(d, old_key, new_key, default_value=None):
    d[new_key] = d.pop(old_key, default_value)

In [8]:
# # 各クラスターの解釈を決定
change_dict_key(cluster_to_words, 1,  'ラーメンに関するワード')
change_dict_key(cluster_to_words, 2, 'スープに関するワード')
change_dict_key(cluster_to_words, 3, 'トッピングに関するワード')
change_dict_key(cluster_to_words, 0, '店内に関するワード')
change_dict_key(cluster_to_words, 4, '店舗に関するワード')
change_dict_key(cluster_to_words, 5, 'その他のワード')

In [9]:
df = pd.DataFrame.from_dict(cluster_to_words, orient="index").T
df.to_csv('data/words_cluster.csv')
df.head(20)

,ラーメンに関するワード,スープに関するワード,トッピングに関するワード,店内に関するワード,店舗に関するワード,その他のワード
0,ラーメン,スープ,チャーシュー,0分,徒歩,さん
1,0円,美味しい,良い,店内,場所,0時
2,こと,醤油,メンマ,カウンター,食べログ,こちら
3,よう,感じ,トッピング,食券,https,訪問
4,これ,濃厚,ネギ,注文,.com,中華そば
5,そう,煮干し,ワンタン,メニュー,出て,人気
6,つけ麺,旨味,食感,券売機,土曜日,行列
7,いい,出汁,野菜,到着,外観,今日
8,ない,味わい,海苔,購入,www,ごちそうさま
9,もの,つけ,具材,店主,沿い,平日


### TF-IDFで文章における特徴的な単語を抽出

In [10]:
from gensim import corpora
from gensim import models

In [11]:
# 口コミからレコメンドに使用する情報を絞る
# 試行錯誤して決定するべし!!
ramen_words = cluster_to_words['ラーメンに関するワード']
soup_words = cluster_to_words['スープに関するワード']
topping_words = cluster_to_words['トッピングに関するワード']

ramen_words.extend(soup_words)
ramen_words.extend(topping_words)
key_words = ramen_words

In [12]:
with open('data/ramen_corpus.txt', 'r', encoding='utf-8') as f:
    trainings = []
    for i, data in enumerate(f):
        # 不要な文字を取り除いて単語を取り出す
        words = data.replace("'", '').replace('[', '').replace(']', '').replace(' ', '').replace('\n', '').split(",")
        trainings.append([i for i in words if i in key_words])

In [13]:
# 単語->id変換の辞書作成
dictionary = corpora.Dictionary(trainings)

In [14]:
# textsをcorpus化
corpus = list(map(dictionary.doc2bow, trainings))

In [15]:
# tfidf modelの生成
test_model = models.TfidfModel(corpus)

In [16]:
# corpusへのモデル適用
corpus_tfidf = test_model[corpus]

In [17]:
# id->単語へ変換
texts_tfidf = []    # id -> 単語表示に変えた文書ごとのTF-IDF
for doc in corpus_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([dictionary[word[0]], word[1]])
    texts_tfidf.append(text_tfidf)

In [18]:
from operator import itemgetter
import pickle

In [19]:
texts_tfidf_sorted_top20 = [] 

#TF-IDF値を高い順に並び替え上位単語20個に絞る。
for i in range(len(texts_tfidf)):
    soted = sorted(texts_tfidf[i], key=itemgetter(1),reverse=True)
    soted_top20 = soted[:20]
    word_list = []
    for k in range(len(soted_top20)):
        word = soted_top20[k][0]
        word_list.append(word)
    texts_tfidf_sorted_top20.append(word_list)

In [20]:
# 結果をデータフレームに追加
store_df = pd.read_csv('data/ramen_store.csv')
review_df = pd.read_csv('data/ramen_review.csv')
ramen_df = pd.merge(store_df, review_df, on='ID')

In [21]:
ramen_df = ramen_df.groupby(['ID', 'name', 'url', 'score'])['review'].apply(list).apply(' '.join).reset_index().sort_values('ID')
ramen_df['texts_tfidf_sorted_top20'] = texts_tfidf_sorted_top20
ramen_df.shape

(500, 6)

In [22]:
df_ramen_texts_tfidf_sorted_top20 = ramen_df[['ID', 'texts_tfidf_sorted_top20']]
df_ramen_texts_tfidf_sorted_top20.head()

,ID,texts_tfidf_sorted_top20
0,1,"[ワンタン, 炭火, 昆布, 海老, 特製, 火炙り, だれ, 藻塩, 焼き豚, 炙り焼, ..."
1,2,"[黒毛和牛, いりこ, 火山, spf, spf豚, ワンタン, 手入れ, 鰹節, チャー,..."
2,3,"[ワンタン, 三つ葉, 特製, 鶏胸肉, 生醤油, 紫蘇, 大葉, うどん, むね, 形成,..."
3,4,"[軍鶏, 替え玉, そば, まぜそば, しゃも, 卵黄, 素地, 燻製, 特製, 醤油漬け,..."
4,5,"[しじみ, シジミ, 琥珀, ワンタン, 雑色, バラ, コハク酸, 地鶏, 特製, 菅野,..."


In [23]:
# 上の結果をダンプ
with open('data/df_ramen_texts_tfidf_sorted_top20', 'wb') as f:
    pickle.dump(df_ramen_texts_tfidf_sorted_top20, f)

In [24]:
ramen_df[['ID', 'name', 'url', 'texts_tfidf_sorted_top20']].to_csv('data/tfidf_top20.csv')